# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from thermolib.thermodynamics.fep import FreeEnergySurface2D, plot_feps
from thermolib.thermodynamics.histogram import Histogram2D
from thermolib.tools import read_wham_input_2D

import numpy as np, matplotlib.pyplot as pp

from molmod.units import *

# Example Elias 1

In [ ]:
fn_fes_ext = '/home/louis/hpc/data/shared/Elias/Elias-Louis/Umbrella_2D/ODealkylation/freefile_no18'
temp = 523*kelvin
fes_ext = FreeEnergySurface2D.from_txt(fn_fes_ext, temp, delimiter='\t', verbose=True)
fes_ext.plot('fes_elias1_ext.png', ncolors=10, lims=[0,100])

In [ ]:
#read temperature and bias potentials from wham_input.txt file
fn_wham = '/home/louis/hpc/data/shared/Elias/Elias-Louis/Umbrella_2D/ODealkylation/metadatafile_no18'
temp_none, biasses, trajectories = read_wham_input_2D(fn_wham, path_template_colvar_fns='%s', stride=10, verbose=True)

## Without error estimation

In [ ]:
bins = [np.arange(0.5,1.5+1/240, 1/240),np.arange(0.0,1.0+1/240,1/240)]
hist = Histogram2D.from_wham(bins, trajectories, biasses, temp, error_estimate=None, verbose=True, convergence=1e-6)

In [ ]:
fes = FreeEnergySurface2D.from_histogram(hist, temp)
fes.set_ref(ref='min')
fes.plot('fes_elias1.png', ncolors=10, lims=[0,100])

In [ ]:
fep_ext = fes_ext.project_difference()
fep_ext.set_ref(ref='min')
fep = fes.project_difference()
fep.set_ref(ref='min')
plot_feps('projected_feps.png', [fep_ext,fep], temp=523*kelvin, 
          labels=['External', 'ThermoLIB'], colors=['k','b'],linestyles=['--', '-'])

In [ ]:
fep_ext = fes_ext.project_average()
fep_ext.set_ref(ref='min')
fep = fes.project_average()
fep.set_ref(ref='min')
plot_feps('projected_feps.png', [fep_ext,fep], temp=523*kelvin, 
          labels=['External', 'ThermoLIB'], colors=['k','b'],linestyles=['--', '-'])

## With error estimation

In [ ]:
bins = [np.arange(0.5,1.5+1/240, 1/240),np.arange(0.0,1.0+1/240,1/240)]
hist = Histogram2D.from_wham(bins, trajectories, biasses, temp, error_estimate='None', verbose=True, convergence=1e-6)

In [ ]:
hist_p = Histogram2D.from_wham(bins, trajectories, biasses, temp, pinit=hist.ps, error_estimate='mle_f', verbose=True, convergence=1e-6)

In [ ]:
fes_f = FreeEnergySurface2D.from_histogram(hist_f, temp)
fes_f.set_ref(ref='min')
fes_f.plot('fes_elias1.png', ncolors=10, lims=[0,100])

In [ ]:
fes_err = FreeEnergySurface2D(fes_f.cv1s.copy(), fes_f.cv2s.copy(), fes_f.fupper-fes_f.flower, fes_f.T)
fes_err.plot('fes_elias1_err.png', ncolors=10, lims=[0,20])

# Example Massimo

In [3]:
fn_fes_ext = '/home/louis/hpc/data/shared/massimo/for_Louis/2D_umbrella/fes_wham.dat'
temp = 573*kelvin
fes_ext = FreeEnergySurface2D.from_txt(fn_fes_ext, temp, delimiter='\t', verbose=True)
fes_ext.plot('fes_ext.png', ncolors=14, lims=[0,140])

CV grid specification
---------------------


NameError: name 'cv1_output__unit' is not defined

In [ ]:
#read temperature and bias potentials from wham_input.txt file
fn_wham = '/home/louis/hpc/data/shared/massimo/for_Louis/2D_umbrella/wham_input_bis.txt'
temp_none, biasses, trajectories = read_wham_input_2D(fn_wham, path_template_colvar_fns='colvar_%s.dat', verbose=True)

First do WHAM without error estimation

In [ ]:
bins = [np.arange(1.305, 3.995+0.02, 0.02), np.arange(-2.495, -0.305+0.02, 0.02)]
hist = Histogram2D.from_wham(bins, trajectories, biasses, temp, error_estimate='mle_f', verbose=True, convergence=1e-3)

In [ ]:
fes = FreeEnergySurface2D.from_histogram(hist, temp)
fes.set_ref(ref='min')
fes.plot('fes.png', ncolors=14, lims=[0,140])

In [ ]:
fes_err = FreeEnergySurface2D(fes.cv1s.copy(), fes.cv2s.copy(), fes.fupper-fes.flower, fes.T)
fes_err.plot('fes_mass_err.png', ncolors=10, lims=[0,20])

In [ ]:
fep = fes.project_average()
fep.plot('fep_mass_err_average.png')

# Example Elias 2

In [ ]:
fn_fes_ext = '/home/louis/hpc/data/shared/Elias/Elias-Louis/Umbrella_2D/CDealkylation/freefile_3108'
temp = 523*kelvin
fes_ext = FreeEnergySurface2D.from_txt(fn_fes_ext, temp, delimiter='\t', verbose=True)
fes_ext.plot('fes_elias2_ext.png', ncolors=12, lims=[0,120])

In [ ]:
#read temperature and bias potentials from wham_input.txt file
fn_wham = '/home/louis/hpc/data/shared/Elias/Elias-Louis/Umbrella_2D/CDealkylation/metadatafile_2D'
temp_none, biasses, trajectories = read_wham_input_2D(fn_wham, path_template_colvar_fns='%s', verbose=True)

In [ ]:
bins = [np.arange(0.0, 0.9+0.9/240, 0.9/240),np.arange(1.4,2.8+1.4/240,1.4/240)]
hist = Histogram2D.from_wham(bins, trajectories, biasses, temp, error_estimate=None, verbose=True, convergence=1e-4, plot_biases=True)

In [ ]:
fes = FreeEnergySurface2D.from_histogram(hist, temp)
fes.set_ref(ref='min')
fes.plot('fes_elias2.png', ncolors=12, lims=[0,120])

In [ ]:
pinit = hist.ps.copy()
hist2 = Histogram2D.from_wham(bins, trajectories, biasses, temp, pinit=pinit, error_estimate=None, verbose=True, convergence=1e-4)

In [ ]:
fes2 = FreeEnergySurface2D.from_histogram(hist2, temp)
fes2.set_ref(ref='min')
fes2.plot('fes_elias2_2.png', ncolors=18, lims=[0,120])

In [ ]:
fes_ext_cleaned = fes_ext.copy()
fes_ext_cleaned.detect_clusters(fn_plot='clusters.png')

In [ ]:
fes_cleaned = fes.copy()
fes_cleaned.detect_clusters(fn_plot='clusters.png')

In [ ]:
fep_ext = fes_ext_cleaned.project_cv1()
fep_ext.process_states(ts_range=[0.3,0.5])
#fep_ext.recollect(np.arange(-1.50,0.50+1/480,1/480))
fep_ext.set_ref(ref='p')
fep = fes_cleaned.project_cv1()
fep.process_states(ts_range=[0.3,0.5])
#fep.recollect(np.arange(-1.50,0.50+1/480,1/480))
fep.set_ref(ref='p')
plot_feps('projected_feps.png', [fep_ext,fep], temp=523*kelvin, 
          labels=['External', 'ThermoLIB'], colors=['k','b'],linestyles=['--', '-'], flims=[-30,120])

In [ ]:
fep_ext2 = fes_ext_cleaned.project_cv2()
fep_ext2.process_states(ts_range=[1.8,2.2])
fep_ext2.set_ref(ref='r')
fep2 = fes_cleaned.project_cv2()
fep2.process_states(ts_range=[1.8,2.2])
fep2.set_ref(ref='r')
plot_feps('projected_feps.png', [fep_ext2,fep2], temp=523*kelvin, 
          labels=['External', 'ThermoLIB'], colors=['k','b'],linestyles=['--', '-'], flims=[-30,120])